In [1]:
import keras

Using TensorFlow backend.
/Users/Jay/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/Jay/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/Jay/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/Jay/opt/anaconda3/lib/python3.7/site-packag

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import mrcnn 
from PIL import Image
from mrcnn.utils import Dataset
from mrcnn.model import MaskRCNN
from mrcnn.config import Config
from mrcnn import model as modellib
from mrcnn import visualize

import numpy as np
from numpy import zeros
from numpy import asarray
import colorsys
import argparse
#import imutils
import random
import os
import time
from matplotlib import pyplot
from matplotlib.patches import Rectangle
from keras.models import load_model
%matplotlib inline
from os import listdir

In [3]:
class PedestrianDataset(Dataset):
    # load the dataset definitions
    def load_dataset(self, dataset_dir, is_train=True):
        
        # Add classes. We have only one class to add.
        self.add_class("dataset", 1, "pedestrians")
        self.add_class("dataset", 2, "riders")
        self.add_class("dataset", 3, "pvp")
        self.add_class("dataset", 4, "ignore")
        self.add_class("dataset", 5, "crowd")
        
        # define data locations for images and annotations
        images_dir = dataset_dir + '/Images/'
        annotations_dir = dataset_dir + '/Annotations/'
        
        # Iterate through all files in the folder to 
        #add class, images and annotaions
        count = 0
        for filename in listdir(images_dir):
            
            # extract image id
            image_id = filename.split(".")[0]
            
            # setting image file
            img_path = images_dir + filename
            
            # setting annotations file
            ann_path = annotations_dir + image_id + '.jpg.txt'
            try:
              file = open(ann_path)
              count+=1
              # skip all images after 150 if we are building the train set
              if is_train and count >= 6001:
                  continue
              # skip all images before 150 if we are building the test/val set
              if not is_train and (count < 6001 or count >= 7501):
                continue 
              # adding images and annotations to dataset
              self.add_image('dataset', image_id=image_id, path=img_path, annotation=ann_path)
            except FileNotFoundError:
              print("File with: "+ ann_path + " not found")
            
            
# extract bounding boxes from an annotation file
    def extract_boxes(self, image_id):
        
        # extract each bounding box
        boxes = list()
        class_ids = list()

        annot_name_path = self.image_info[image_id]['annotation']
        img_name_path = self.image_info[image_id]['path']
        print(img_name_path)
        count = 0
        with open(annot_name_path) as file: 
          for line in file: 
              if count == 0:
                  count+=1
              else:
                  class_ids.append(line.strip().split(" ")[0])
                  coors = line.strip().split(" ")[1:5]
                  coors = [int(x) for x in coors]
                  boxes.append(coors)
        
        im = Image.open(img_name_path)
        width, height = im.size

        return class_ids, boxes, width, height
    # load the masks for an image
    """Generate instance masks for an image.
       Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
     """
    def load_mask(self, image_id):
        # get details of image
        info = self.image_info[image_id]
        print(info)

        # define anntation  file location
        # path = info['annotation']
        
        # load bounding boxes
        class_ids, boxes, w, h = self.extract_boxes(image_id)
       
        # create one array for all masks, each on a different channel
        masks = zeros([h, w, len(boxes)], dtype='uint8')
        
        # create masks
        for i in range(len(boxes)):
            box = boxes[i]
            row_s, row_e = box[1], box[3]
            col_s, col_e = box[0], box[2]
            masks[row_s:row_e, col_s:col_e, i] = 1

        return masks, asarray(class_ids, dtype='int32')
    
    # load an image reference
    """Return the path of the image."""
    def image_reference(self, image_id):
        info = self.image_info[image_id]
        print(info)
        return info['path']

In [4]:
# prepare train set
train_set = PedestrianDataset()
train_set.load_dataset("WiderPerson-2", is_train=True)
train_set.prepare()

# prepare test/val set
test_set = PedestrianDataset()
test_set.load_dataset("WiderPerson-2", is_train=False)
test_set.prepare()

print("Train: %d" % len(train_set.image_ids))
print("Test: %d" % len(test_set.image_ids))

File with: WiderPerson-2/Annotations/.jpg.txt not found
File with: WiderPerson-2/Annotations/000043.jpg.txt not found
File with: WiderPerson-2/Annotations/000047.jpg.txt not found
File with: WiderPerson-2/Annotations/000052.jpg.txt not found
File with: WiderPerson-2/Annotations/000053.jpg.txt not found
File with: WiderPerson-2/Annotations/000054.jpg.txt not found
File with: WiderPerson-2/Annotations/000055.jpg.txt not found
File with: WiderPerson-2/Annotations/000056.jpg.txt not found
File with: WiderPerson-2/Annotations/000057.jpg.txt not found
File with: WiderPerson-2/Annotations/000058.jpg.txt not found
File with: WiderPerson-2/Annotations/000064.jpg.txt not found
File with: WiderPerson-2/Annotations/000066.jpg.txt not found
File with: WiderPerson-2/Annotations/000067.jpg.txt not found
File with: WiderPerson-2/Annotations/000068.jpg.txt not found
File with: WiderPerson-2/Annotations/000069.jpg.txt not found
File with: WiderPerson-2/Annotations/000070.jpg.txt not found
File with: Wid

File with: WiderPerson-2/Annotations/004663.jpg.txt not found
File with: WiderPerson-2/Annotations/004667.jpg.txt not found
File with: WiderPerson-2/Annotations/004681.jpg.txt not found
File with: WiderPerson-2/Annotations/004686.jpg.txt not found
File with: WiderPerson-2/Annotations/004698.jpg.txt not found
File with: WiderPerson-2/Annotations/004699.jpg.txt not found
File with: WiderPerson-2/Annotations/004700.jpg.txt not found
File with: WiderPerson-2/Annotations/004705.jpg.txt not found
File with: WiderPerson-2/Annotations/004706.jpg.txt not found
File with: WiderPerson-2/Annotations/004707.jpg.txt not found
File with: WiderPerson-2/Annotations/004708.jpg.txt not found
File with: WiderPerson-2/Annotations/004709.jpg.txt not found
File with: WiderPerson-2/Annotations/004713.jpg.txt not found
File with: WiderPerson-2/Annotations/004717.jpg.txt not found
File with: WiderPerson-2/Annotations/004725.jpg.txt not found
File with: WiderPerson-2/Annotations/004730.jpg.txt not found
File wit

File with: WiderPerson-2/Annotations/008095.jpg.txt not found
File with: WiderPerson-2/Annotations/008098.jpg.txt not found
File with: WiderPerson-2/Annotations/008108.jpg.txt not found
File with: WiderPerson-2/Annotations/008111.jpg.txt not found
File with: WiderPerson-2/Annotations/008113.jpg.txt not found
File with: WiderPerson-2/Annotations/008119.jpg.txt not found
File with: WiderPerson-2/Annotations/008133.jpg.txt not found
File with: WiderPerson-2/Annotations/008145.jpg.txt not found
File with: WiderPerson-2/Annotations/008148.jpg.txt not found
File with: WiderPerson-2/Annotations/008166.jpg.txt not found
File with: WiderPerson-2/Annotations/008171.jpg.txt not found
File with: WiderPerson-2/Annotations/008237.jpg.txt not found
File with: WiderPerson-2/Annotations/008500.jpg.txt not found
File with: WiderPerson-2/Annotations/008501.jpg.txt not found
File with: WiderPerson-2/Annotations/008505.jpg.txt not found
File with: WiderPerson-2/Annotations/008515.jpg.txt not found
File wit

File with: WiderPerson-2/Annotations/011048.jpg.txt not found
File with: WiderPerson-2/Annotations/011049.jpg.txt not found
File with: WiderPerson-2/Annotations/011052.jpg.txt not found
File with: WiderPerson-2/Annotations/011055.jpg.txt not found
File with: WiderPerson-2/Annotations/011062.jpg.txt not found
File with: WiderPerson-2/Annotations/011064.jpg.txt not found
File with: WiderPerson-2/Annotations/011065.jpg.txt not found
File with: WiderPerson-2/Annotations/011066.jpg.txt not found
File with: WiderPerson-2/Annotations/011072.jpg.txt not found
File with: WiderPerson-2/Annotations/011082.jpg.txt not found
File with: WiderPerson-2/Annotations/011084.jpg.txt not found
File with: WiderPerson-2/Annotations/011089.jpg.txt not found
File with: WiderPerson-2/Annotations/011092.jpg.txt not found
File with: WiderPerson-2/Annotations/011093.jpg.txt not found
File with: WiderPerson-2/Annotations/011096.jpg.txt not found
File with: WiderPerson-2/Annotations/011097.jpg.txt not found
File wit

File with: WiderPerson-2/Annotations/014699.jpg.txt not found
File with: WiderPerson-2/Annotations/014702.jpg.txt not found
File with: WiderPerson-2/Annotations/014705.jpg.txt not found
File with: WiderPerson-2/Annotations/014706.jpg.txt not found
File with: WiderPerson-2/Annotations/014709.jpg.txt not found
File with: WiderPerson-2/Annotations/014719.jpg.txt not found
File with: WiderPerson-2/Annotations/014720.jpg.txt not found
File with: WiderPerson-2/Annotations/014721.jpg.txt not found
File with: WiderPerson-2/Annotations/014725.jpg.txt not found
File with: WiderPerson-2/Annotations/014727.jpg.txt not found
File with: WiderPerson-2/Annotations/014736.jpg.txt not found
File with: WiderPerson-2/Annotations/014737.jpg.txt not found
File with: WiderPerson-2/Annotations/014740.jpg.txt not found
File with: WiderPerson-2/Annotations/014743.jpg.txt not found
File with: WiderPerson-2/Annotations/014746.jpg.txt not found
File with: WiderPerson-2/Annotations/014749.jpg.txt not found
File wit

...

File with: WiderPerson-2/Annotations/010590.jpg.txt not found
File with: WiderPerson-2/Annotations/010592.jpg.txt not found
File with: WiderPerson-2/Annotations/010602.jpg.txt not found
File with: WiderPerson-2/Annotations/010618.jpg.txt not found
File with: WiderPerson-2/Annotations/010619.jpg.txt not found
File with: WiderPerson-2/Annotations/010621.jpg.txt not found
File with: WiderPerson-2/Annotations/010626.jpg.txt not found
File with: WiderPerson-2/Annotations/010637.jpg.txt not found
File with: WiderPerson-2/Annotations/010638.jpg.txt not found
File with: WiderPerson-2/Annotations/010640.jpg.txt not found
File with: WiderPerson-2/Annotations/010648.jpg.txt not found
File with: WiderPerson-2/Annotations/010650.jpg.txt not found
File with: WiderPerson-2/Annotations/010652.jpg.txt not found
File with: WiderPerson-2/Annotations/010653.jpg.txt not found
File with: WiderPerson-2/Annotations/010656.jpg.txt not found
File with: WiderPerson-2/Annotations/010660.jpg.txt not found
File wit

File with: WiderPerson-2/Annotations/013245.jpg.txt not found
File with: WiderPerson-2/Annotations/013250.jpg.txt not found
File with: WiderPerson-2/Annotations/013256.jpg.txt not found
File with: WiderPerson-2/Annotations/013258.jpg.txt not found
File with: WiderPerson-2/Annotations/013261.jpg.txt not found
File with: WiderPerson-2/Annotations/013262.jpg.txt not found
File with: WiderPerson-2/Annotations/013269.jpg.txt not found
File with: WiderPerson-2/Annotations/013275.jpg.txt not found
File with: WiderPerson-2/Annotations/013276.jpg.txt not found
File with: WiderPerson-2/Annotations/013277.jpg.txt not found
File with: WiderPerson-2/Annotations/013279.jpg.txt not found
File with: WiderPerson-2/Annotations/013280.jpg.txt not found
File with: WiderPerson-2/Annotations/013282.jpg.txt not found
File with: WiderPerson-2/Annotations/013287.jpg.txt not found
File with: WiderPerson-2/Annotations/013293.jpg.txt not found
File with: WiderPerson-2/Annotations/013295.jpg.txt not found
File wit

File with: WiderPerson-2/Annotations/015051.jpg.txt not found
File with: WiderPerson-2/Annotations/015073.jpg.txt not found
File with: WiderPerson-2/Annotations/015100.jpg.txt not found
File with: WiderPerson-2/Annotations/015115.jpg.txt not found
File with: WiderPerson-2/Annotations/015125.jpg.txt not found
File with: WiderPerson-2/Annotations/015130.jpg.txt not found
File with: WiderPerson-2/Annotations/015165.jpg.txt not found
File with: WiderPerson-2/Annotations/015166.jpg.txt not found
File with: WiderPerson-2/Annotations/015187.jpg.txt not found
File with: WiderPerson-2/Annotations/015202.jpg.txt not found
File with: WiderPerson-2/Annotations/015214.jpg.txt not found
File with: WiderPerson-2/Annotations/015221.jpg.txt not found
File with: WiderPerson-2/Annotations/015251.jpg.txt not found
File with: WiderPerson-2/Annotations/015257.jpg.txt not found
File with: WiderPerson-2/Annotations/015260.jpg.txt not found
File with: WiderPerson-2/Annotations/015286.jpg.txt not found
File wit

File with: WiderPerson-2/Annotations/016836.jpg.txt not found
File with: WiderPerson-2/Annotations/016838.jpg.txt not found
File with: WiderPerson-2/Annotations/016842.jpg.txt not found
File with: WiderPerson-2/Annotations/016845.jpg.txt not found
File with: WiderPerson-2/Annotations/016848.jpg.txt not found
File with: WiderPerson-2/Annotations/016850.jpg.txt not found
File with: WiderPerson-2/Annotations/016851.jpg.txt not found
File with: WiderPerson-2/Annotations/016855.jpg.txt not found
File with: WiderPerson-2/Annotations/016856.jpg.txt not found
File with: WiderPerson-2/Annotations/016863.jpg.txt not found
File with: WiderPerson-2/Annotations/016868.jpg.txt not found
File with: WiderPerson-2/Annotations/016870.jpg.txt not found
File with: WiderPerson-2/Annotations/016872.jpg.txt not found
File with: WiderPerson-2/Annotations/016876.jpg.txt not found
File with: WiderPerson-2/Annotations/016881.jpg.txt not found
File with: WiderPerson-2/Annotations/016884.jpg.txt not found
File wit

File with: WiderPerson-2/Annotations/019110.jpg.txt not found
File with: WiderPerson-2/Annotations/019112.jpg.txt not found
File with: WiderPerson-2/Annotations/019120.jpg.txt not found
File with: WiderPerson-2/Annotations/019129.jpg.txt not found
File with: WiderPerson-2/Annotations/019135.jpg.txt not found
File with: WiderPerson-2/Annotations/019145.jpg.txt not found
File with: WiderPerson-2/Annotations/019159.jpg.txt not found
File with: WiderPerson-2/Annotations/019164.jpg.txt not found
File with: WiderPerson-2/Annotations/019165.jpg.txt not found
File with: WiderPerson-2/Annotations/019176.jpg.txt not found
File with: WiderPerson-2/Annotations/019184.jpg.txt not found
File with: WiderPerson-2/Annotations/019196.jpg.txt not found
File with: WiderPerson-2/Annotations/019197.jpg.txt not found
File with: WiderPerson-2/Annotations/019207.jpg.txt not found
File with: WiderPerson-2/Annotations/019216.jpg.txt not found
File with: WiderPerson-2/Annotations/019220.jpg.txt not found
File wit

In [5]:
class myMaskRCNNConfig(Config):
    # give the configuration a recognizable name
    NAME = "MaskRCNN_config"
 
    # set the number of GPUs to use along with the number of images
    # per GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
 
    # number of classes (we would normally add +1 for the background)
     # 5 + BG
    NUM_CLASSES = 5+1
   
   # Number of training steps per epoch
    STEPS_PER_EPOCH = len(train_set.image_ids)
    VALIDATION_STEPS = len(test_set.image_ids)
    
    # Learning rate
    LEARNING_RATE=0.001
    
    # Skip detections with < 65% confidence
    DETECTION_MIN_CONFIDENCE = 0.65
    
    # setting Max ground truth instances
    MAX_GT_INSTANCES=1000

config = myMaskRCNNConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.65
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                18
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INST

In [8]:
print("Loading Mask R-CNN model...")
model = modellib.MaskRCNN(mode="training", config=config, model_dir='./')
#load the weights for COCO
model.load_weights('./mask_rcnn_maskrcnn_config_0008.h5', 
                   by_name=True, 
                   exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",  "mrcnn_bbox", "mrcnn_mask"])



Loading Mask R-CNN model...


In [9]:
print(model.find_last())

FileNotFoundError: [Errno 2] Could not find weight files in ./maskrcnn_config20200502T0104

In [ ]:
print("Loading Mask R-CNN model...")
model = modellib.MaskRCNN(mode="training", config=config, model_dir='./')

#load the weights for COCO
model.load_weights('./mask_rcnn_coco.h5', 
                   by_name=True, 
                   exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",  "mrcnn_bbox", "mrcnn_mask"])

# train heads with higher lr to speedup the learning
model.train(train_set, test_set, learning_rate=2*config.LEARNING_RATE, epochs=3, layers="heads")
history = model.keras_model.history.history

Loading Mask R-CNN model...






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
box_ind is deprecated, use box_indices instead

Starting at epoch 0. LR=0.02

Checkpoint Path: ./maskrcnn_config20200417T0104/mask_rcnn_maskrcnn_config_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3   

/Users/Jay/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/Users/Jay/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/Users/Jay/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/Users/Jay/opt/anaconda3/lib/python3.7/site-packages/keras/engine/training_generator.py:47: UserWarning: Using a gener



Epoch 1/3
{'id': '003356', 'source': 'dataset', 'path': 'WiderPerson-2/Images/003356.jpg', 'annotation': 'WiderPerson-2/Annotations/003356.jpg.txt'}
WiderPerson-2/Images/003356.jpg
{'id': '001848', 'source': 'dataset', 'path': 'WiderPerson-2/Images/001848.jpg', 'annotation': 'WiderPerson-2/Annotations/001848.jpg.txt'}
WiderPerson-2/Images/001848.jpg
{'id': '001848', 'source': 'dataset', 'path': 'WiderPerson-2/Images/001848.jpg', 'annotation': 'WiderPerson-2/Annotations/001848.jpg.txt'}
{'id': '001848', 'source': 'dataset', 'path': 'WiderPerson-2/Images/001848.jpg', 'annotation': 'WiderPerson-2/Annotations/001848.jpg.txt'}
WiderPerson-2/Images/001848.jpg
WiderPerson-2/Images/001848.jpg
{'id': '003356', 'source': 'dataset', 'path': 'WiderPerson-2/Images/003356.jpg', 'annotation': 'WiderPerson-2/Annotations/003356.jpg.txt'}
WiderPerson-2/Images/003356.jpg
{'id': '001848', 'source': 'dataset', 'path': 'WiderPerson-2/Images/001848.jpg', 'annotation': 'WiderPerson-2/Annotations/001848.jpg.

WiderPerson-2/Images/003391.jpg
{'id': '003391', 'source': 'dataset', 'path': 'WiderPerson-2/Images/003391.jpg', 'annotation': 'WiderPerson-2/Annotations/003391.jpg.txt'}
WiderPerson-2/Images/003391.jpg
{'id': '003391', 'source': 'dataset', 'path': 'WiderPerson-2/Images/003391.jpg', 'annotation': 'WiderPerson-2/Annotations/003391.jpg.txt'}
WiderPerson-2/Images/003391.jpg
{'id': '000491', 'source': 'dataset', 'path': 'WiderPerson-2/Images/000491.jpg', 'annotation': 'WiderPerson-2/Annotations/000491.jpg.txt'}
WiderPerson-2/Images/000491.jpg
{'id': '000491', 'source': 'dataset', 'path': 'WiderPerson-2/Images/000491.jpg', 'annotation': 'WiderPerson-2/Annotations/000491.jpg.txt'}
{'id': '000491', 'source': 'dataset', 'path': 'WiderPerson-2/Images/000491.jpg', 'annotation': 'WiderPerson-2/Annotations/000491.jpg.txt'}
WiderPerson-2/Images/000491.jpg
WiderPerson-2/Images/000491.jpg
{'id': '003954', 'source': 'dataset', 'path': 'WiderPerson-2/Images/003954.jpg', 'annotation': 'WiderPerson-2/Ann

WiderPerson-2/Images/000350.jpg
{'id': '003846', 'source': 'dataset', 'path': 'WiderPerson-2/Images/003846.jpg', 'annotation': 'WiderPerson-2/Annotations/003846.jpg.txt'}
WiderPerson-2/Images/003846.jpg
{'id': '001438', 'source': 'dataset', 'path': 'WiderPerson-2/Images/001438.jpg', 'annotation': 'WiderPerson-2/Annotations/001438.jpg.txt'}
WiderPerson-2/Images/001438.jpg
{'id': '003138', 'source': 'dataset', 'path': 'WiderPerson-2/Images/003138.jpg', 'annotation': 'WiderPerson-2/Annotations/003138.jpg.txt'}
WiderPerson-2/Images/003138.jpg
{'id': '003893', 'source': 'dataset', 'path': 'WiderPerson-2/Images/003893.jpg', 'annotation': 'WiderPerson-2/Annotations/003893.jpg.txt'}
WiderPerson-2/Images/003893.jpg
{'id': '004040', 'source': 'dataset', 'path': 'WiderPerson-2/Images/004040.jpg', 'annotation': 'WiderPerson-2/Annotations/004040.jpg.txt'}
WiderPerson-2/Images/004040.jpg
{'id': '001424', 'source': 'dataset', 'path': 'WiderPerson-2/Images/001424.jpg', 'annotation': 'WiderPerson-2/Ann

...

WiderPerson-2/Images/000444.jpg
{'id': '003494', 'source': 'dataset', 'path': 'WiderPerson-2/Images/003494.jpg', 'annotation': 'WiderPerson-2/Annotations/003494.jpg.txt'}
WiderPerson-2/Images/003494.jpg
{'id': '000631', 'source': 'dataset', 'path': 'WiderPerson-2/Images/000631.jpg', 'annotation': 'WiderPerson-2/Annotations/000631.jpg.txt'}
WiderPerson-2/Images/000631.jpg
{'id': '003766', 'source': 'dataset', 'path': 'WiderPerson-2/Images/003766.jpg', 'annotation': 'WiderPerson-2/Annotations/003766.jpg.txt'}
WiderPerson-2/Images/003766.jpg
{'id': '003300', 'source': 'dataset', 'path': 'WiderPerson-2/Images/003300.jpg', 'annotation': 'WiderPerson-2/Annotations/003300.jpg.txt'}
WiderPerson-2/Images/003300.jpg
{'id': '001804', 'source': 'dataset', 'path': 'WiderPerson-2/Images/001804.jpg', 'annotation': 'WiderPerson-2/Annotations/001804.jpg.txt'}
WiderPerson-2/Images/001804.jpg
WiderPerson-2/Images/004414.jpg
{'id': '004414', 'source': 'dataset', 'path': 'WiderPerson-2/Images/004414.jpg', 

WiderPerson-2/Images/000444.jpg
{'id': '004099', 'source': 'dataset', 'path': 'WiderPerson-2/Images/004099.jpg', 'annotation': 'WiderPerson-2/Annotations/004099.jpg.txt'}
WiderPerson-2/Images/004099.jpg
{'id': '003483', 'source': 'dataset', 'path': 'WiderPerson-2/Images/003483.jpg', 'annotation': 'WiderPerson-2/Annotations/003483.jpg.txt'}
WiderPerson-2/Images/003483.jpg
{'id': '001860', 'source': 'dataset', 'path': 'WiderPerson-2/Images/001860.jpg', 'annotation': 'WiderPerson-2/Annotations/001860.jpg.txt'}
WiderPerson-2/Images/001860.jpg
{'id': '003776', 'source': 'dataset', 'path': 'WiderPerson-2/Images/003776.jpg', 'annotation': 'WiderPerson-2/Annotations/003776.jpg.txt'}
WiderPerson-2/Images/003776.jpg
{'id': '004018', 'source': 'dataset', 'path': 'WiderPerson-2/Images/004018.jpg', 'annotation': 'WiderPerson-2/Annotations/004018.jpg.txt'}
WiderPerson-2/Images/004018.jpg
{'id': '001815', 'source': 'dataset', 'path': 'WiderPerson-2/Images/001815.jpg', 'annotation': 'WiderPerson-2/Ann

WiderPerson-2/Images/000599.jpg
{'id': '003364', 'source': 'dataset', 'path': 'WiderPerson-2/Images/003364.jpg', 'annotation': 'WiderPerson-2/Annotations/003364.jpg.txt'}
WiderPerson-2/Images/003364.jpg
{'id': '001860', 'source': 'dataset', 'path': 'WiderPerson-2/Images/001860.jpg', 'annotation': 'WiderPerson-2/Annotations/001860.jpg.txt'}
WiderPerson-2/Images/001860.jpg
{'id': '000954', 'source': 'dataset', 'path': 'WiderPerson-2/Images/000954.jpg', 'annotation': 'WiderPerson-2/Annotations/000954.jpg.txt'}
WiderPerson-2/Images/000954.jpg
{'id': '003483', 'source': 'dataset', 'path': 'WiderPerson-2/Images/003483.jpg', 'annotation': 'WiderPerson-2/Annotations/003483.jpg.txt'}
WiderPerson-2/Images/003483.jpg
{'id': '004087', 'source': 'dataset', 'path': 'WiderPerson-2/Images/004087.jpg', 'annotation': 'WiderPerson-2/Annotations/004087.jpg.txt'}
WiderPerson-2/Images/004087.jpg
{'id': '003231', 'source': 'dataset', 'path': 'WiderPerson-2/Images/003231.jpg', 'annotation': 'WiderPerson-2/Ann

WiderPerson-2/Images/000997.jpg
{'id': '004046', 'source': 'dataset', 'path': 'WiderPerson-2/Images/004046.jpg', 'annotation': 'WiderPerson-2/Annotations/004046.jpg.txt'}
WiderPerson-2/Images/004046.jpg
{'id': '003661', 'source': 'dataset', 'path': 'WiderPerson-2/Images/003661.jpg', 'annotation': 'WiderPerson-2/Annotations/003661.jpg.txt'}
WiderPerson-2/Images/003661.jpg
{'id': '003364', 'source': 'dataset', 'path': 'WiderPerson-2/Images/003364.jpg', 'annotation': 'WiderPerson-2/Annotations/003364.jpg.txt'}
WiderPerson-2/Images/003364.jpg
{'id': '003337', 'source': 'dataset', 'path': 'WiderPerson-2/Images/003337.jpg', 'annotation': 'WiderPerson-2/Annotations/003337.jpg.txt'}
WiderPerson-2/Images/003337.jpg
{'id': '000556', 'source': 'dataset', 'path': 'WiderPerson-2/Images/000556.jpg', 'annotation': 'WiderPerson-2/Annotations/000556.jpg.txt'}
WiderPerson-2/Images/000556.jpg
{'id': '004087', 'source': 'dataset', 'path': 'WiderPerson-2/Images/004087.jpg', 'annotation': 'WiderPerson-2/Ann

WiderPerson-2/Images/003508.jpg
{'id': '004416', 'source': 'dataset', 'path': 'WiderPerson-2/Images/004416.jpg', 'annotation': 'WiderPerson-2/Annotations/004416.jpg.txt'}
WiderPerson-2/Images/004416.jpg
{'id': '004044', 'source': 'dataset', 'path': 'WiderPerson-2/Images/004044.jpg', 'annotation': 'WiderPerson-2/Annotations/004044.jpg.txt'}
WiderPerson-2/Images/004044.jpg
{'id': '004167', 'source': 'dataset', 'path': 'WiderPerson-2/Images/004167.jpg', 'annotation': 'WiderPerson-2/Annotations/004167.jpg.txt'}
WiderPerson-2/Images/004167.jpg
{'id': '003532', 'source': 'dataset', 'path': 'WiderPerson-2/Images/003532.jpg', 'annotation': 'WiderPerson-2/Annotations/003532.jpg.txt'}
WiderPerson-2/Images/003532.jpg
{'id': '003936', 'source': 'dataset', 'path': 'WiderPerson-2/Images/003936.jpg', 'annotation': 'WiderPerson-2/Annotations/003936.jpg.txt'}
WiderPerson-2/Images/003936.jpg
{'id': '004006', 'source': 'dataset', 'path': 'WiderPerson-2/Images/004006.jpg', 'annotation': 'WiderPerson-2/Ann

In [10]:
import time
model_path = 'PedestrianModel/mask_rcnn_maskrcnn_config_0008.h5'
model.load_weights(model_path)

In [11]:
print(train_set.class_ids)

[0 1 2 3 4 5]


In [ ]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
#Loading the model in the inference mode
model = modellib.MaskRCNN(mode="inference", config=config, model_dir='./')
# loading the trained weights o the custom dataset
model.load_weights(model_path, by_name=True)
                   ## , exclude=["mrcnn_class_logits", "mrcnn_bbox_fc","mrcnn_mask"])
img = load_img("WiderPerson-2/Images/004040.jpg")
img = img_to_array(img)
# detecting objects in the image
result= model.detect([img])
result


Instructions for updating:
Use `tf.cast` instead.


In [ ]:
history.keys()

In [ ]:
epochs = range(1,len(next(iter(history.values())))+1)
pd.DataFrame(history, index=epochs)

In [ ]:
plt.figure(figsize=(17,5))

plt.subplot(131)
plt.plot(epochs, history["loss"], label="Train loss")
plt.plot(epochs, history["val_loss"], label="Valid loss")
plt.legend()
plt.subplot(132)
plt.plot(epochs, history["mrcnn_class_loss"], label="Train class ce")
plt.plot(epochs, history["val_mrcnn_class_loss"], label="Valid class ce")
plt.legend()
plt.subplot(133)
plt.plot(epochs, history["mrcnn_bbox_loss"], label="Train box loss")
plt.plot(epochs, history["val_mrcnn_bbox_loss"], label="Valid box loss")
plt.legend()

plt.show()

In [ ]:
dataset = test_set
fig = plt.figure(figsize=(10, 30))

for i in range(5):

    image_id = random.choice(dataset.image_ids)
    
    original_image, image_meta, gt_class_id, gt_bbox, gt_mask =\
        modellib.load_image_gt(test_set, config,image_id, use_mini_mask=False)
    
    print(original_image.shape)
    plt.subplot(6, 2, 2*i + 1)
    visualize.display_instances(original_image, gt_bbox, gt_mask, gt_class_id, 
                                dataset.class_names,ax=fig.axes[-1])
    
    plt.subplot(6, 2, 2*i + 2)
    results = model.detect([original_image]) #, verbose=1)
    r = results[0]
    visualize.display_instances(original_image, r['rois'], r['masks'], r['class_ids'], 
                                dataset.class_names, r['scores'], ax=fig.axes[-1])

In [ ]:
Y_pred = model.predict_generator(val_generator, self.VALIDATION_STEPS)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(validation_generator.classes, y_pred))

In [ ]:
image_id = 2
image, image_meta, gt_class_id, gt_bbox, gt_mask = \
modellib.load_image_gt(test_set, config, image_id, use_mini_mask=False)
info = test_set.image_info[image_id]
print("image ID: {}.{} ({}) {}".format(info["source"], info["id"], image_id, 
                                       test_set.image_reference(image_id)))
# Run object detection
results = model.detect([image], verbose=1)
# Display results

r = results[0]
visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                            test_set.class_names, r['scores'], 
                            title="Predictions")